# Progress report 1 notebook

## Reload libraries each time, since we’re tinkering with them

In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load libraries

In [26]:
from xml.dom import pulldom  # parse input XML
from xml.dom.minidom import Document  # construct output XML
import numpy as np
import pandas as pd
import regex as re
from cyr2phon import cyr2phon, utility  # custom package

## Class and variables for parsing input XML

In [27]:
class Stack(list):  # keep track of open nodes while constructing XML output
    def push(self, item):
        self.append(item)

    def peek(self):  
        return self[-1]


open_elements = Stack()
WS_RE = re.compile(r'\s+')  # normalize white space in output

## Function to parse the XML

Returns a list of lists, with stanza number, line number, and `<line>` element for each line

In [28]:
def process(input_xml):
    stanzaNo = 0
    lineNo = 0
    inline = 0  # flag to control behavior inside and outside lines
    result = []  # array of arrays, one per line, with stanzaNo, lineNo, and serialized XML
    doc = pulldom.parse(input_xml)
    for event, node in doc:
        if event == pulldom.START_ELEMENT and node.localName == 'stanza':
            stanzaNo = node.getAttribute("stanzaNo")
        elif event == pulldom.START_ELEMENT and node.localName == 'line':
            d = Document()  # each line is an output XML document
            open_elements.push(d)  # document node
            lineNo = node.getAttribute("lineNo")
            inline = 1  # we’re inside a line
            open_elements.peek().appendChild(node)  # add as child of current node in output tree
            open_elements.push(node)  # keep track of open elements
        elif event == pulldom.END_ELEMENT and node.localName == 'line':
            inline = 0  # when we finish our work here, we’ll no longer be inside a line
            open_elements.pop()  # line is finished
            # serialize XML, strip declaration, rewrite &quot; entity as character
            result.append([int(stanzaNo), int(lineNo),
                WS_RE.sub(" " ,
                open_elements.pop().toxml().replace('<?xml version="1.0" ?>', '').replace('&quot;', '"'))])
        elif event == pulldom.START_ELEMENT and node.localName == 'stress':
            open_elements.peek().appendChild(node)  # add as child of current node in output tree
            open_elements.push(node)  # keep track of open elements
        elif event == pulldom.END_ELEMENT and node.localName == 'stress':
            open_elements.pop()  # stress element is finished
        elif event == pulldom.CHARACTERS and inline:  # keep text only inside lines
            t = d.createTextNode(node.data)
            open_elements.peek().appendChild(t)
    return result

## Parse the XML into an array of arrays

In [29]:
with open("data_samples/eo1.xml") as f:
    all_lines = process(f)
all_lines[:5]  # take a look

[[1,
  1,
  '<line lineNo="001">"Мой дядя самых честных пр<stress>а</stress>вил,</line>'],
 [1,
  2,
  '<line lineNo="002">Когда не в шутку занем<stress>о</stress>г,</line>'],
 [1, 3, '<line lineNo="003">Он уважать себя заст<stress>а</stress>вил</line>'],
 [1, 4, '<line lineNo="004">И лучше выдумать не м<stress>о</stress>г.</line>'],
 [1, 5, '<line lineNo="005">Его пример другим на<stress>у</stress>ка;</line>']]

## Write the data into a dataframe

In [30]:
df = pd.DataFrame(all_lines, columns=["StanzaNo", "LineNo", "Text"])
df.head(5)

,StanzaNo,LineNo,Text
0,1,1,"<line lineNo=""001"">""Мой дядя самых честных пр<..."
1,1,2,"<line lineNo=""002"">Когда не в шутку занем<stre..."
2,1,3,"<line lineNo=""003"">Он уважать себя заст<stress..."
3,1,4,"<line lineNo=""004"">И лучше выдумать не м<stres..."
4,1,5,"<line lineNo=""005"">Его пример другим на<stress..."


## Transliterate all lines and save in new column

### Notes

1. Because only the last stress in the line is marked, the phonetic representation of all words except the last is incorrect. That doesn’t matter for the analysis of end rhyme.
1. Words in foreign languages are not being treated specially, and are therefore usually phonetically incorrect. That *does* matter for the analysis of end rhyme. Deal with it later, first by excluding those lines (revise the XML parsing to record that information), and eventually by phoneticizing them correctly.

In [31]:
trans_vec = np.vectorize(cyr2phon.transliterate)
df["Phonetic"] = trans_vec(df["Text"])
df.head()  # take a look

,StanzaNo,LineNo,Text,Phonetic
0,1,1,"<line lineNo=""001"">""Мой дядя самых честных пр<...",maJ DiDi samix Čistnix prAVil
1,1,2,"<line lineNo=""002"">Когда не в шутку занем<stre...",kagda Nifšutku zaNimOk
2,1,3,"<line lineNo=""003"">Он уважать себя заст<stress...",an uvažaT SiBi zastAVil
3,1,4,"<line lineNo=""004"">И лучше выдумать не м<stres...",iluČši vidumaT NimOk
4,1,5,"<line lineNo=""005"">Его пример другим на<stress...",Jiva pRiMir druGim naUka


## Write the rhyme word into a new column

In [32]:
df["RhymeWord"] = df["Phonetic"].str.split().str[-1]
df.head()  # take a look

,StanzaNo,LineNo,Text,Phonetic,RhymeWord
0,1,1,"<line lineNo=""001"">""Мой дядя самых честных пр<...",maJ DiDi samix Čistnix prAVil,prAVil
1,1,2,"<line lineNo=""002"">Когда не в шутку занем<stre...",kagda Nifšutku zaNimOk,zaNimOk
2,1,3,"<line lineNo=""003"">Он уважать себя заст<stress...",an uvažaT SiBi zastAVil,zastAVil
3,1,4,"<line lineNo=""004"">И лучше выдумать не м<stres...",iluČši vidumaT NimOk,NimOk
4,1,5,"<line lineNo=""005"">Его пример другим на<stress...",Jiva pRiMir druGim naUka,naUka


## Syllabify rhyme word and write into new column

In [23]:
syllabify_vec = np.vectorize(utility.syllabify)
df["RhymeWord"] = syllabify_vec(df["RhymeWord"])
df.head()

,StanzaNo,LineNo,Text,Phonetic,RhymeWord
0,1,1,"<line lineNo=""001"">""Мой дядя самых честных пр<...",maJ DiDi samix Čistnix prAVil,prA---Vil
1,1,2,"<line lineNo=""002"">Когда не в шутку занем<stre...",kagda Nifšutku zaNimOk,za---Ni---mOk
2,1,3,"<line lineNo=""003"">Он уважать себя заст<stress...",an uvažaT SiBi zastAVil,za---stA---Vil
3,1,4,"<line lineNo=""004"">И лучше выдумать не м<stres...",iluČši vidumaT NimOk,Ni---mOk
4,1,5,"<line lineNo=""005"">Его пример другим на<stress...",Jiva pRiMir druGim naUka,na---U---ka


In [24]:
for i in df["RhymeWord"][:5]:
    print(i)

prA---Vil
za---Ni---mOk
za---stA---Vil
Ni---mOk
na---U---ka


## Find rhyme pairs

Each fourteen-line stanza in this poem has the same regular rhyme scheme: **aBaBccDDeFFeGG**. We can use this regularity to find lines that rhyme by matching the line numbers, creating a gold standard that we can use to test our (eventual) analytic identification of rhyme.

## (Resume here)

Todo:

1. decompose words into syllables
1. identify rhyme zone
1. decompose syllables into segments
1. decompose segments into phonetic features
1. build table of rhymes
1. identify imperfect rhymes and describe and analyze at segment and feature level